In [1]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import tflearn
import random
import json

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


curses is not supported on this machine (please install/reinstall curses for an optimal experience)
Instructions for updating:
Colocations handled automatically by placer.


In [9]:
#print(tf.__version__)

1.13.2


In [5]:
#pip install tflearn --user

Note: you may need to restart the kernel to use updated packages.


In [3]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)
    
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Welcome here,How can I help you?',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you soon, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'who',
   'patterns': ['Who are you?', 'What can you do?'],
   'responses': ['I am a Chatbot powered by RGNIIPM, I can help you out with your queries']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'timimg',
   'patterns': ['What time can I take training?',
    'What are the training hours?'],
   'responses': ['Our hours are 10.00 AM to 5.30 PM',
    'Training Timing: - 10.00 AM to 5.30 PM']},
  {'tag': 'location',
   'patterns': ['What is 

In [4]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:

# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

32 documents
11 classes ['faculties', 'goodbye', 'greeting', 'location', 'needforawareness', 'payments', 'thanks', 'timimg', 'trainings', 'who', 'whocanjoin']
59 unique stemmed words ["'s", ',', 'acceiv', 'address', 'anyon', 'ar', 'avail', 'aw', 'bye', 'can', 'card', 'cash', 'conduc', 'credit', 'day', 'do', 'doe', 'facul', 'for', 'good', 'goodby', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'join', 'lat', 'loc', 'mastercard', 'method', 'nee', 'of', 'on', 'org', 'pay', 'program', 'provid', 'reach', 'rgniipm', 'see', 'should', 'tak', 'thank', 'that', 'the', 'ther', 'tim', 'train', 'typ', 'what', 'wher', 'who', 'why', 'wil', 'yo', 'you']


In [6]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [7]:
# resetting underlying graph data
tf.reset_default_graph()

# Building neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('saved_models/model.tflearn')

Training Step: 3999  | total loss: 0.02428 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.02428 - acc: 0.9992 -- iter: 24/32
Training Step: 4000  | total loss: 0.02324 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.02324 - acc: 0.9993 -- iter: 32/32
--
INFO:tensorflow:C:\Users\Hp Pavilion\Chatbot-nltk\saved_models\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [9]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [10]:
# load the saved model
model.load('saved_models/model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Hp Pavilion\Chatbot-nltk\saved_models\model.tflearn


In [11]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [12]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [13]:
response('hi')

Hi there, how can I help?


In [14]:
response('what can you do?')

I am a Chatbot powered by RGNIIPM, I can help you out with your queries


In [15]:
response('available training')

RGNIIPM conducts Training, Awareness programs relating to Intellectual Property Rights i.e. Patents, Designs, Trademarks, Geographical Indications & Copyrights


In [16]:
response('bye')

Bye! Come back again soon.
